# MDATP - Use query from Github Repository

In [1]:
## This will run a script to import the required modules and credentials stored in the "credential" file
$scriptPath = "~/Notebooks/PowerShell/custom-modules/import.ps1"
. $scriptPath  # this will run the script

The below script needs to be able to find the current output cell; this is an easy method to get it.


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

## Pull raw query from Github
Be sure to provide the <font color=Green><b>RAW Content</b></font> and not a the github link of the source

In [2]:
$rawGithubLink = "https://raw.githubusercontent.com/microsoft/Microsoft-threat-protection-Hunting-Queries/master/Discovery/URL%20Detection.txt"
$Response = Invoke-WebRequest -Method GET -URI $rawGithubLink
$body = (@{'Query' = $Response.Content}) | ConvertTO-JSON;
$Response.Content

// This query finds network communication to specific URL
// Please note that in line #7 it filters RemoteUrl using has operator, which looks for a "whole term" and runs faster.
// Example: RemoteUrl has "microsoft" matches "www.microsoft.com" but not "microsoftonline.com"
let partialRemoteUrlToDetect = "microsoft.com"; // Change this to a URL you'd like to find machines connecting to
DeviceNetworkEvents  
| where Timestamp > ago(7d)
and RemoteUrl has partialRemoteUrlToDetect // Can be changed to "contains" operator as explained above
| project Timestamp, DeviceName, DeviceId, ReportId
| top 100 by Timestamp desc



## Put your Hunting Query here

In [7]:
## Enter your query here below $Query
$body = (@{'Query' = $Response.Content}) | ConvertTO-JSON;
$body

{
  "Query": "// This query finds network communication to specific URL\n// Please note that in line #7 it filters RemoteUrl using has operator, which looks for a \"whole term\" and runs faster.\n// Example: RemoteUrl has \"microsoft\" matches \"www.microsoft.com\" but not \"microsoftonline.com\"\nlet partialRemoteUrlToDetect = \"microsoft.com\"; // Change this to a URL you'd like to find machines connecting to\nDeviceNetworkEvents  \n| where Timestamp > ago(7d)\nand RemoteUrl has partialRemoteUrlToDetect // Can be changed to \"contains\" operator as explained above\n| project Timestamp, DeviceName, DeviceId, ReportId\n| top 100 by Timestamp desc\n"
}


## Query the MDATP API Grab report schema 

In [8]:
$URI = "https://api.securitycenter.windows.com/api/advancedqueries/run"
$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" }
$authHeader
$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $authHeader -body $body -ErrorAction Stop)
$Result


Name                           Value
----                           -----
Content-Type                   application/json
Authorization                  Bearer 
Accept                         application/json
Invoke-RestMethod: 
Line |
   7 |  $Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $authHea …
     |             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Response status code does not indicate success: 401 (Unauthorized).



## Parse report

In [7]:
$includeColumns = @('Timestamp','DeviceName','ActionType','RemoteIP')
$Result.Results | Select-Object -Property $includeColumns


Timestamp             DeviceName        ActionType RemoteIP
---------             ----------        ---------- --------
6/24/2020 7:12:32 PM  win10-oobe-test              
6/24/2020 7:12:30 PM  win10-oobe-test              
6/24/2020 7:12:29 PM  win10-oobe-test              
6/24/2020 7:02:17 PM  jingtoso-desktop1            
6/24/2020 6:48:40 PM  win10-oobe-test              
6/24/2020 6:40:39 PM  win10-oobe-test              
6/24/2020 6:38:11 PM  jingtoso-desktop1            
6/24/2020 6:18:44 PM  win10-oobe-test              
6/24/2020 6:18:24 PM  win10-oobe-test              
6/24/2020 6:18:23 PM  win10-oobe-test              
6/24/2020 6:18:16 PM  win10-oobe-test              
6/24/2020 6:18:15 PM  win10-oobe-test              
6/24/2020 6:13:13 PM  win10-oobe-test              
6/24/2020 5:58:32 PM  win10-oobe-test              
6/24/2020 5:57:34 PM  win10-oobe-test              
6/24/2020 5:36:50 PM  jing-lab                     
6/24/2020 5:24:09 PM  win10-oobe-test          